In [14]:
import numpy as np
import pandas as pd
import pickle
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.metrics import log_loss,accuracy_score

1. **MUST RUN 01-Data_Preprocessing/01-preprocessing_code.py AND 02-Feature_Engineering/01-fe_code.py BEFORE**

### We have the following dataset :

In [15]:
# load objects/X_DATASET.pkl and objects/Y_DATASET.pkl
X = pd.read_pickle('../objects/X_DATASET.pkl')
Y = pd.read_pickle('../objects/Y_DATASET.pkl')

In [16]:
X

,age,aliq_at,aliq_mat,ami_126d,at_be,at_gr1,at_me,at_turnover,be_gr1a,be_me,...,taccruals_at,taccruals_ni,tangibility,tax_gr1a,turnover_126d,turnover_var_126d,z_score,zero_trades_126d,zero_trades_21d,zero_trades_252d
0,132,0.985818,0.059472,0.000026,1.260918,0.651395,0.042021,1.028183,0.310450,0.033325,...,0.132759,0.932668,0.287709,0.018174,0.003022,0.336775,43.032362,0.004157,0.005002,0.003226
2781,784,0.636280,0.419246,0.000106,3.002603,0.003136,0.982853,1.561359,-0.007330,0.327334,...,-0.070334,-1.350656,0.571700,0.030609,0.003256,0.451566,3.381417,0.003881,0.003926,0.003549
79192,156,1.078982,0.497954,0.000215,12.300384,0.289454,0.750931,0.212139,0.029724,0.061049,...,-0.011451,-0.503262,0.858820,0.007016,0.005098,0.404042,1.087062,0.002407,0.003124,0.001720
79017,96,0.766219,0.152166,0.012623,1.311442,0.022729,0.131301,0.662498,0.058250,0.100119,...,-0.170568,-11.313364,0.751451,-0.003988,0.007496,1.475238,9.327025,0.001392,0.000767,0.001469
136801,444,1000000.000000,1000000.000000,0.000583,21.847603,0.141701,4.562675,0.090603,0.005307,0.208841,...,1000000.000000,1000000.000000,1000000.000000,0.000656,0.003407,1.192832,1000000.000000,0.003737,0.005589,0.004025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142960,875,0.494857,0.258809,0.000051,2.561048,-0.019929,0.599322,0.790637,0.076776,0.234014,...,-0.007516,-0.074240,0.615093,0.010144,0.008145,1.452597,2.313842,0.004100,0.005428,0.004865
164883,335,0.440289,0.204966,0.000043,1.369123,0.041082,0.509132,1.101150,0.053765,0.371868,...,-0.040519,-0.331965,0.445832,-0.002505,0.006820,0.484558,6.825448,0.004906,0.004621,0.004572
56626,467,0.456242,0.073563,0.000047,7.338437,0.106965,0.194904,0.601361,0.050950,0.026559,...,-0.077962,-0.812357,0.310944,-0.004359,0.006439,0.423431,3.953374,0.005186,0.004429,0.005337
103667,347,0.232903,0.142937,0.000458,2.143037,-0.028275,0.774278,1.205285,0.030875,0.361299,...,0.020596,0.472952,0.235576,0.002153,0.004803,0.504602,3.420703,0.006556,0.005650,0.006052


In [17]:
Y

,stock_exret,target,prediction,probability,t1,t1_index,weight_attr
0,0.018070,1,1.0,0.133210,2000-01-31,2000-01-03,0.018070
2781,0.069806,1,-1.0,0.602130,2000-01-31,2000-01-03,0.069806
79192,-0.066192,-1,-1.0,0.484446,2000-01-31,2000-01-03,0.066192
79017,0.825067,1,-1.0,0.881480,2000-01-31,2000-01-03,0.825067
136801,0.093419,1,1.0,0.819652,2000-01-31,2000-01-03,0.093419
...,...,...,...,...,...,...,...
142960,0.015598,1,-1.0,0.281077,2023-12-29,2023-12-01,0.015598
164883,0.068874,1,1.0,0.664075,2023-12-29,2023-12-01,0.068874
56626,0.110382,1,1.0,0.217701,2023-12-29,2023-12-01,0.110382
103667,0.000132,1,1.0,0.770597,2023-12-29,2023-12-01,0.000132


In [18]:
X['t1_index'] = Y['t1_index']
X.reset_index(inplace=True)
X.set_index(['t1_index', 'index'], inplace=True)
X

age         aliq_at        aliq_mat  ami_126d      at_be  \
t1_index   index                                                              
2000-01-03 0       132        0.985818        0.059472  0.000026   1.260918   
           2781    784        0.636280        0.419246  0.000106   3.002603   
           79192   156        1.078982        0.497954  0.000215  12.300384   
           79017    96        0.766219        0.152166  0.012623   1.311442   
           136801  444  1000000.000000  1000000.000000  0.000583  21.847603   
...                ...             ...             ...       ...        ...   
2023-12-01 142960  875        0.494857        0.258809  0.000051   2.561048   
           164883  335        0.440289        0.204966  0.000043   1.369123   
           56626   467        0.456242        0.073563  0.000047   7.338437   
           103667  347        0.232903        0.142937  0.000458   2.143037   
           172784  936        0.502752        0.448166  0.000064   2.224959   

                     at_gr1     at_me  at_turnover   be_gr1a     be_me  ...  \
t1_index   index                                                        ...   
2000-01-03 0       0.651395  0.042021     1.028183  0.310450  0.033325  ...   
           2781    0.003136  0.982853     1.561359 -0.007330  0.327334  ...   
           79192   0.289454  0.750931     0.212139  0.029724  0.061049  ...   
           79017   0.022729  0.131301     0.662498  0.058250  0.100119  ...   
           136801  0.141701  4.562675     0.090603  0.005307  0.208841  ...   
...                     ...       ...          ...       ...       ...  ...   
2023-12-01 142960 -0.019929  0.599322     0.790637  0.076776  0.234014  ...   
           164883  0.041082  0.509132     1.101150  0.053765  0.371868  ...   
           56626   0.106965  0.194904     0.601361  0.050950  0.026559  ...   
           103667 -0.028275  0.774278     1.205285  0.030875  0.361299  ...   
           172784  0.033296  1.989022     0.229889 -0.000261  0.893959  ...   

                     taccruals_at    taccruals_ni     tangibility  tax_gr1a  \
t1_index   index                                                              
2000-01-03 0             0.132759        0.932668        0.287709  0.018174   
           2781         -0.070334       -1.350656        0.571700  0.030609   
           79192        -0.011451       -0.503262        0.858820  0.007016   
           79017        -0.170568      -11.313364        0.751451 -0.003988   
           136801  1000000.000000  1000000.000000  1000000.000000  0.000656   
...                           ...             ...             ...       ...   
2023-12-01 142960       -0.007516       -0.074240        0.615093  0.010144   
           164883       -0.040519       -0.331965        0.445832 -0.002505   
           56626        -0.077962       -0.812357        0.310944 -0.004359   
           103667        0.020596        0.472952        0.235576  0.002153   
           172784       -0.053687       -2.701708        0.587483  0.000052   

                   turnover_126d  turnover_var_126d         z_score  \
t1_index   index                                                      
2000-01-03 0            0.003022           0.336775       43.032362   
           2781         0.003256           0.451566        3.381417   
           79192        0.005098           0.404042        1.087062   
           79017        0.007496           1.475238        9.327025   
           136801       0.003407           1.192832  1000000.000000   
...                          ...                ...             ...   
2023-12-01 142960       0.008145           1.452597        2.313842   
           164883       0.006820           0.484558        6.825448   
           56626        0.006439           0.423431        3.953374   
           103667       0.004803           0.504602        3.420703   
           172784       0.007469           0.391692        1.081265   

           

### X contains the feautes given and ADDED BY US

In [19]:
Y = Y.reset_index()
Y.set_index(['t1_index', 'index'], inplace=True)
Y


stock_exret  target  prediction  probability          t1  \
t1_index   index                                                              
2000-01-03 0          0.018070       1         1.0     0.133210  2000-01-31   
           2781       0.069806       1        -1.0     0.602130  2000-01-31   
           79192     -0.066192      -1        -1.0     0.484446  2000-01-31   
           79017      0.825067       1        -1.0     0.881480  2000-01-31   
           136801     0.093419       1         1.0     0.819652  2000-01-31   
...                        ...     ...         ...          ...         ...   
2023-12-01 142960     0.015598       1        -1.0     0.281077  2023-12-29   
           164883     0.068874       1         1.0     0.664075  2023-12-29   
           56626      0.110382       1         1.0     0.217701  2023-12-29   
           103667     0.000132       1         1.0     0.770597  2023-12-29   
           172784     0.042407       1         1.0     0.936564  2023-12-29   

                   weight_attr  
t1_index   index                
2000-01-03 0          0.018070  
           2781       0.069806  
           79192      0.066192  
           79017      0.825067  
           136801     0.093419  
...                        ...  
2023-12-01 142960     0.015598  
           164883     0.068874  
           56626      0.110382  
           103667     0.000132  
           172784     0.042407  

[172785 rows x 6 columns]

### Currently, in $Y$,

*prediction* and *probability* are randomly generated, those are the columns
we must fill for the investment perdiod (2008-) we our true probabilities and predictions.

## WE WANT TO PREDICT TARGET, which is simply the sign of stock_exret

## Weight_attr IS SO IMPORANT, PLEASE DON'T IGNORE 

As Uday did in feature importance, this weight_attr must be rescaled for the 
training period. Right now, it is scaled for the whole dataset, you must reajust
it for the training period such that the suym of the weight = Size of the training period

*= X_train.shape[0]/X_train['weight_attr'].sum()

Uday did it I believe

Refer to section 4.6 if you have any doubts

# Hyperparameter Tuning

## Method

The method is explained in the FIAM instructions, please refer to pages\
6 and 7, sections **Performance evaluation** AND **Training Procedures**\
(especially training procedures)

As they mention it in the doc, they provided an implementation of the method,

penalized_linear_hackathon.py

In [20]:
# Necessary imports
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import PredefinedSplit, RandomizedSearchCV
from skopt import BayesSearchCV
from skopt.space import Integer, Real, Categorical
from scipy.stats import randint, uniform


In [21]:


stock_vars = ['ret_1_0', 'prc_highprc_252d', 'seas_1_1na', 'rmax5_21d']  # List of features
stock_vars = [
    # Historical Returns & Momentum Indicators
    'ret_12_1',            # 12-month return excluding the last month (momentum indicator)
    'ret_60_12',           # Long-term momentum (5-year minus 1-year returns)

    # Valuation Metrics
    'market_equity',        # Market capitalization (size factor)
    'intrinsic_value',      # Fundamental valuation metric, useful for identifying mispriced stocks
    'be_me',                # Book-to-market ratio (value factor)

    # Profitability Indicators
    'ebit_sale',            # Operating profitability ratio (quality factor)
    # 'eps_actual',           # Actual earnings per share (earnings indicator)

    # Growth Indicators
    'sale_gr1',             # 1-year sales growth (growth factor)
    'emp_gr1',              # 1-year employee growth (operational growth efficiency)

    # Financial Health & Leverage
    'netdebt_me',           # Net debt-to-market equity (leverage risk indicator)
    'z_score',              # Altman Z-score (financial distress risk indicator)

    # Risk & Volatility Measures
    'beta_60m',             # 5-year beta (systematic risk measure)
    'ivol_capm_21d',        # Idiosyncratic volatility (measure of unsystematic risk)
    'rvol_21d',             # 21-day return volatility (short-term risk measure)

    # Liquidity & Market Microstructure
    'dolvol_126d',          # Dollar trading volume over 126 days (liquidity factor)
    'zero_trades_252d',     # Frequency of zero-return days over a year (illiquidity proxy)

    # Quality & Profitability Measures
    'f_score',              # Piotroski F-score (financial health score)
    'qmj_prof',             # Quality-minus-junk (profitability quality factor)

    # Behavioral & Mispricing Indicators
    'mispricing_perf',      # Performance-based mispricing indicator
    'age'                   # Age of the firm (younger firms tend to be riskier but with potential growth)
]


In [22]:

tgt_var = 'target'  # Target variable

starting = pd.to_datetime("20000101", format="%Y%m%d")
counter = 0
pred_out = pd.DataFrame()




X.index = pd.MultiIndex.from_tuples(
    [(pd.to_datetime(t1_index), other_index) for t1_index, other_index in X.index]
)
Y.index = pd.MultiIndex.from_tuples(
    [(pd.to_datetime(t1_index), other_index) for t1_index, other_index in Y.index]
)


In [23]:


# Estimation with expanding window
while (starting + pd.DateOffset(years=11 + counter)) <= pd.to_datetime("20240101", format="%Y%m%d"):
    # For testing purposes
    # if counter == 1:
    #     break

    cutoff = [
        starting,
        starting + pd.DateOffset(years=8 + counter),  # Training set end date
        starting + pd.DateOffset(years=10 + counter),  # Validation set end date
        starting + pd.DateOffset(years=11 + counter),  # Test set end date
    ]

    print(f"Train Start: {cutoff[0]}, Train End: {cutoff[1]}, Val Start: {cutoff[1]}, Val End: {cutoff[2]}, Test Start: {cutoff[2]}, Test End: {cutoff[3]}")

    # Cut the sample into training, validation, and testing sets
    X_train = X[(X.index.get_level_values(0) >= cutoff[0]) & (X.index.get_level_values(0) < cutoff[1])]
    X_validate = X[(X.index.get_level_values(0) >= cutoff[1]) & (X.index.get_level_values(0) < cutoff[2])]
    X_test = X[(X.index.get_level_values(0) >= cutoff[2]) & (X.index.get_level_values(0) < cutoff[3])]

    Y_train = Y[(Y.index.get_level_values(0) >= cutoff[0]) & (Y.index.get_level_values(0) < cutoff[1])]
    Y_validate = Y[(Y.index.get_level_values(0) >= cutoff[1]) & (Y.index.get_level_values(0) < cutoff[2])]
    Y_test = Y[(Y.index.get_level_values(0) >= cutoff[2]) & (Y.index.get_level_values(0) < cutoff[3])]

    # # Adjust sample weights
    # Y_train['weight_attr'] *= Y_train.shape[0] / Y_train['weight_attr'].sum()
    # Y_validate['weight_attr'] *= Y_validate.shape[0] / Y_validate['weight_attr'].sum()
    # Y_test['weight_attr'] *= Y_test.shape[0] / Y_test['weight_attr'].sum()


    X_train_vals = X_train[stock_vars].values
    X_validate_vals = X_validate[stock_vars].values
    X_test_vals = X_test[stock_vars].values

    X_train_val = np.vstack([X_train_vals, X_validate_vals])
    Y_train_val = pd.concat([Y_train, Y_validate])

    # Create test_fold 
    test_fold = np.concatenate([
        np.full(len(X_train_vals), -1),  # training set indices
        np.zeros(len(X_validate_vals))   # validation set indices
    ])
    ps = PredefinedSplit(test_fold)


    base_rf = RandomForestClassifier(
        criterion="entropy",
        bootstrap=False,
        class_weight="balanced_subsample"
    )


    bagging_clf = BaggingClassifier(
        estimator=base_rf,
        oob_score=True,
        n_jobs=-1
    )

 
    param_distributions = {
    'estimator__n_estimators': randint(10, 1000),
    'estimator__max_depth': randint(5, 50),
    'estimator__min_samples_split': randint(2, 10),
    'estimator__min_samples_leaf': randint(1, 5),
    'estimator__max_features': ['sqrt', 'log2'],  
    'n_estimators': randint(10, 100),  
    'max_samples': uniform(0.1, 1.0),  
    'max_features': uniform(0.1, 1.0)  
    }

    # Define the optimizer
    optimizer = RandomizedSearchCV(
    bagging_clf,
    param_distributions=param_distributions,
    n_iter=1,  
    cv=ps,      # Use predefined split
    n_jobs=-1,
    verbose=2,
    random_state=42
    )   


    optimizer.fit(
        X_train_val,
        Y_train_val[tgt_var].values,
        **{'sample_weight': Y_train_val['weight_attr'].values}
    )


    best_estimator = optimizer.best_estimator_


    best_estimator.fit(
        X_train_val,
        Y_train_val[tgt_var].values,
        sample_weight=Y_train_val['weight_attr'].values
    )

    # Predict on test set
    prob = best_estimator.predict_proba(X_test_vals)
    score_ = -log_loss(Y_test[tgt_var].values, prob, sample_weight=Y_test["weight_attr"].values, labels=best_estimator.classes_)
    print("Log Loss on Test Set:", score_)

    # Store predictions in Y_test
    Y_test['prediction'] = best_estimator.predict(X_test_vals)
    Y_test['probability'] = prob.max(axis=1)

    # print("Predictions on Test Set:")
    # print(Y_test['prediction'])
    # print("Probabilities on Test Set:")
    # print(Y_test['probability'])
    # print("Prediction Probabilities:")
    # print(prob)

 
    pred_out = pred_out.append(Y_test[['prediction', 'probability']])
    pred_out.to_csv("../objects/predictions.csv")


    counter += 1


Train Start: 2000-01-01 00:00:00, Train End: 2008-01-01 00:00:00, Val Start: 2008-01-01 00:00:00, Val End: 2010-01-01 00:00:00, Test Start: 2010-01-01 00:00:00, Test End: 2011-01-01 00:00:00
Fitting 1 folds for each of 1 candidates, totalling 1 fits
[CV] END estimator__max_depth=43, estimator__max_features=log2, estimator__min_samples_leaf=1, estimator__min_samples_split=8, estimator__n_estimators=116, max_features=0.8796910002727693, max_samples=0.696850157946487, n_estimators=92; total time= 9.9min


KeyboardInterrupt: 

## THE CLASSIFIER

You can use one of the following. For clarification, refer to **6.4**

I believe clf2 is the best one

### This is from my deprado repo, *ch_06.ipynb*

```python
# Libraries
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Dataset
X, Y = f_ch8.getTestData(n_features=40, n_informative=3, n_redundant=30, n_samples=10000)
Y = Y["bin"]

# usual RF
clf0 = RandomForestClassifier(
    n_estimators=1_000,  # 1_000 trees
    class_weight="balanced_subsample",  # prevent minority class from being ignored
    criterion="entropy"  # information gain
)

# Ensemble of estimators with base estimator as a decision tree
clf1 = DecisionTreeClassifier(
    criterion="entropy",  # information gain
    max_features="sqrt",  # sqrt(n_features) to force diversity among trees
    class_weight="balanced"  # prevent minority class from being ignored
)
clf1 = BaggingClassifier(
    estimator=clf1,  # base estimator
    n_estimators=1_000,  # 1_000 trees
    max_samples=0.6,  # average uniqueness
    max_features=1.0  # all features for bagging
)

# Bagging classifier on RF where max_samples is set to average uniqueness
clf2 = RandomForestClassifier(
    n_estimators=1,  # 1 tree
    criterion="entropy",  # information gain
    bootstrap=False,  # no bootstrap
    class_weight="balanced_subsample"  # prevent minority class from being ignored
)

clf2 = BaggingClassifier(
    estimator=clf2,  # base estimator
    n_estimators=1_000,  # 1_000 trees
    max_samples=1.0,  # average uniqueness
    max_features=1.0  # all features for bagging
)
```

## Hyperparameter Tunning

### Here is why the weights are so important



Refer to *9.4* , you can use accuracy, but negative log likelihood is suggested,\
since it takes into account the probability of the prediction.

For the hyperparaemtes, i believe you guys can figure out better than me what
to set

## Please be aware of the bugs mentioned in

- 7.5 BUGS IN SKLEARN’S CROSS-VALIDATION
- SNIPPET 9.2 AN ENHANCED PIPELINE CLASS


## I'M AVAILABLE ALL DAY, PLEASE ASK ME ANYTHING, LET'S GOOOOO

In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from joblib import Parallel, delayed

# ... (your existing imports and data loading code)

def process_time_window(counter):
    # Define your starting point
    starting = pd.to_datetime("2000-01-01")
    
    # Calculate cutoff dates
    cutoff = [
        starting,
        starting + pd.DateOffset(years=8 + counter),  # Training set end date
        starting + pd.DateOffset(years=10 + counter),  # Validation set end date
        starting + pd.DateOffset(years=11 + counter),  # Test set end date
    ]

    print(f"Processing Time Window {counter}:")
    print(f"Train Start: {cutoff[0]}, Train End: {cutoff[1]}, Val Start: {cutoff[1]}, Val End: {cutoff[2]}, Test Start: {cutoff[2]}, Test End: {cutoff[3]}")

    # Split the data
    X_train = X[(X.index.get_level_values(0) >= cutoff[0]) & (X.index.get_level_values(0) < cutoff[1])]
    X_validate = X[(X.index.get_level_values(0) >= cutoff[1]) & (X.index.get_level_values(0) < cutoff[2])]
    X_test = X[(X.index.get_level_values(0) >= cutoff[2]) & (X.index.get_level_values(0) < cutoff[3])]

    Y_train = Y[(Y.index.get_level_values(0) >= cutoff[0]) & (Y.index.get_level_values(0) < cutoff[1])]
    Y_validate = Y[(Y.index.get_level_values(0) >= cutoff[1]) & (Y.index.get_level_values(0) < cutoff[2])]
    Y_test = Y[(Y.index.get_level_values(0) >= cutoff[2]) & (Y.index.get_level_values(0) < cutoff[3])]

    X_train_vals = X_train[stock_vars].values
    X_validate_vals = X_validate[stock_vars].values
    X_test_vals = X_test[stock_vars].values

    X_train_val = np.vstack([X_train_vals, X_validate_vals])
    Y_train_val = pd.concat([Y_train, Y_validate])

    # Create test_fold 
    test_fold = np.concatenate([
        np.full(len(X_train_vals), -1),  # training set indices
        np.zeros(len(X_validate_vals))   # validation set indices
    ])
    ps = PredefinedSplit(test_fold)

    base_rf = RandomForestClassifier(
        criterion="entropy",
        bootstrap=False,
        class_weight="balanced_subsample",
        n_jobs=1  # Limit to 1 core per estimator
    )

    bagging_clf = BaggingClassifier(
        estimator=base_rf,
        oob_score=True,
        n_jobs=1  # Limit to 1 core per BaggingClassifier
    )

    param_distributions = {
        'estimator__n_estimators': randint(10, 100),
        'estimator__max_depth': randint(5, 50),
        'estimator__min_samples_split': randint(2, 10),
        'estimator__min_samples_leaf': randint(1, 5),
        'estimator__max_features': ['sqrt', 'log2'],  
        'n_estimators': randint(10, 100),  
        'max_samples': uniform(0.1, 1.0),  
        'max_features': uniform(0.1, 1.0)  
    }

    optimizer = RandomizedSearchCV(
        bagging_clf,
        param_distributions=param_distributions,
        n_iter=10,  # Increase iterations for better search
        cv=ps,      # Use predefined split
        n_jobs=1,   # Limit to 1 core to prevent nested parallelism
        verbose=2,
        random_state=42
    )

    optimizer.fit(
        X_train_val,
        Y_train_val[tgt_var].values,
        sample_weight=Y_train_val['weight_attr'].values
    )

    best_estimator = optimizer.best_estimator_

    best_estimator.fit(
        X_train_val,
        Y_train_val[tgt_var].values,
        sample_weight=Y_train_val['weight_attr'].values
    )

    # Predict on test set
    prob = best_estimator.predict_proba(X_test_vals)
    score_ = -log_loss(Y_test[tgt_var].values, prob, sample_weight=Y_test["weight_attr"].values, labels=best_estimator.classes_)
    print("Log Loss on Test Set:", score_)

    # Store predictions in Y_test
    Y_test['prediction'] = best_estimator.predict(X_test_vals)
    Y_test['probability'] = prob.max(axis=1)

    return Y_test[['prediction', 'probability']]

# Prepare the list of counters
starting = pd.to_datetime("2000-01-01")
end_date = pd.to_datetime("2024-01-01")
max_counter = ((end_date - starting).days // 365) - 11  # Adjust based on your data
counters_list = list(range(max_counter))

import multiprocessing
print(f"Number of CPU cores available: {multiprocessing.cpu_count()}")

# Run in parallel
n_parallel_jobs = 4  # Adjust this number based on your CPU cores
results = Parallel(n_jobs=n_parallel_jobs)(
    delayed(process_time_window)(counter)
    for counter in counters_list
)

# Combine results
pred_out = pd.concat(results)
pred_out.to_csv("../objects/predictions.csv")


Processing Time Window 0:
Train Start: 2000-01-01 00:00:00, Train End: 2008-01-01 00:00:00, Val Start: 2008-01-01 00:00:00, Val End: 2010-01-01 00:00:00, Test Start: 2010-01-01 00:00:00, Test End: 2011-01-01 00:00:00
Processing Time Window 2:
Train Start: 2000-01-01 00:00:00, Train End: 2010-01-01 00:00:00, Val Start: 2010-01-01 00:00:00, Val End: 2012-01-01 00:00:00, Test Start: 2012-01-01 00:00:00, Test End: 2013-01-01 00:00:00
Processing Time Window 3:
Train Start: 2000-01-01 00:00:00, Train End: 2011-01-01 00:00:00, Val Start: 2011-01-01 00:00:00, Val End: 2013-01-01 00:00:00, Test Start: 2013-01-01 00:00:00, Test End: 2014-01-01 00:00:00
Processing Time Window 1:
Train Start: 2000-01-01 00:00:00, Train End: 2009-01-01 00:00:00, Val Start: 2009-01-01 00:00:00, Val End: 2011-01-01 00:00:00, Test Start: 2011-01-01 00:00:00, Test End: 2012-01-01 00:00:00
Fitting 1 folds for each of 10 candidates, totalling 10 fits
Fitting 1 folds for each of 10 candidates, totalling 10 fits
Fitting 1 

/Users/paulkelendji/miniconda3/envs/financial_math/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
1 fits failed out of a total of 10.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/paulkelendji/miniconda3/envs/financial_math/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/paulkelendji/miniconda3/envs/financial_math/lib/python3.8/site-packages/sklearn/base.py", line 1145, in wrapper
    estimator._validate_params()
  File "/Users/paulkelendji/miniconda3/envs/financial_math/lib/python3.8/site-pa

Log Loss on Test Set: -0.6816387098299219


/var/folders/j0/8v9qcjfx15g5ftmsy5n0qrq80000gn/T/ipykernel_28953/1475686741.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/var/folders/j0/8v9qcjfx15g5ftmsy5n0qrq80000gn/T/ipykernel_28953/1475686741.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Processing Time Window 4:
Train Start: 2000-01-01 00:00:00, Train End: 2012-01-01 00:00:00, Val Start: 2012-01-01 00:00:00, Val End: 2014-01-01 00:00:00, Test Start: 2014-01-01 00:00:00, Test End: 2015-01-01 00:00:00
Fitting 1 folds for each of 10 candidates, totalling 10 fits
[CV] END estimator__max_depth=44, estimator__max_features=log2, estimator__min_samples_leaf=2, estimator__min_samples_split=7, estimator__n_estimators=15, max_features=0.3079416628681888, max_samples=0.6677003278199914, n_estimators=72; total time= 3.7min
[CV] END estimator__max_depth=22, estimator__max_features=log2, estimator__min_samples_leaf=4, estimator__min_samples_split=3, estimator__n_estimators=83, max_features=0.9948273504276488, max_samples=0.6978999788110851, n_estimators=57; total time=17.9min


...

[CV] END estimator__max_depth=26, estimator__max_features=sqrt, estimator__min_samples_leaf=4, estimator__min_samples_split=2, estimator__n_estimators=58, max_features=0.6247746602583891, max_samples=0.49986097171525545, n_estimators=69; total time=26.6min
[CV] END estimator__max_depth=20, estimator__max_features=sqrt, estimator__min_samples_leaf=2, estimator__min_samples_split=7, estimator__n_estimators=56, max_features=0.7183860093330873, max_samples=0.4824619912671627, n_estimators=73; total time=22.7min
[CV] END estimator__max_depth=7, estimator__max_features=sqrt, estimator__min_samples_leaf=3, estimator__min_samples_split=8, estimator__n_estimators=30, max_features=0.550499251969543, max_samples=0.11326496115986653, n_estimators=98; total time= 2.4min
[CV] END estimator__max_depth=22, estimator__max_features=log2, estimator__min_samples_leaf=4, estimator__min_samples_split=3, estimator__n_estimators=83, max_features=0.9948273504276488, max_samples=0.6978999788110851, n_estimators

/Users/paulkelendji/miniconda3/envs/financial_math/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
1 fits failed out of a total of 10.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/paulkelendji/miniconda3/envs/financial_math/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/paulkelendji/miniconda3/envs/financial_math/lib/python3.8/site-packages/sklearn/base.py", line 1145, in wrapper
    estimator._validate_params()
  File "/Users/paulkelendji/miniconda3/envs/financial_math/lib/python3.8/site-pa

[CV] END estimator__max_depth=18, estimator__max_features=log2, estimator__min_samples_leaf=2, estimator__min_samples_split=2, estimator__n_estimators=99, max_features=0.19767211400638388, max_samples=0.7842330265121569, n_estimators=69; total time=19.5min
Log Loss on Test Set: -0.6633656394823757


/var/folders/j0/8v9qcjfx15g5ftmsy5n0qrq80000gn/T/ipykernel_28953/1475686741.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/var/folders/j0/8v9qcjfx15g5ftmsy5n0qrq80000gn/T/ipykernel_28953/1475686741.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


[CV] END estimator__max_depth=11, estimator__max_features=log2, estimator__min_samples_leaf=1, estimator__min_samples_split=9, estimator__n_estimators=56, max_features=0.27336465350777206, max_samples=0.49106060757324077, n_estimators=59; total time= 7.8min
[CV] END estimator__max_depth=44, estimator__max_features=log2, estimator__min_samples_leaf=2, estimator__min_samples_split=7, estimator__n_estimators=15, max_features=0.3079416628681888, max_samples=0.6677003278199914, n_estimators=72; total time= 7.5min
